# Constrained Probabilistic Matrix Factorization

["about article of PMF - by Ruslan and Andriy"](https://papers.nips.cc/paper/2007/file/d7322ed717dedf1eb4e6e52a37ea7bcd-Paper.pdf)

In [401]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [402]:
"setting parameters"
dimension = 10
stdv = 0.05
stdv_Y = 0.5
stdv_V = 0.5
stdv_W = 0.5
learning_rate = 0.005
momentum = 0.2
parameters = {}
epoch = 100

# parameters['moment_Y'] = moment_Y
# parameters['moment_V'] = moment_V
# parameters['momnet_W'] = moment_W

### Import_data
The code 

In [403]:
def import_data(path):
    data = pd.read_csv(path, sep='\t', names=['User', 'Item', 'Rating', 'Timestamp'], header=None)
    data = data.drop('Timestamp', axis=1)
    
    data_pure = data.drop_duplicates(['User','Item'], keep = 'last')
    
    data_sort = data_pure.sort_values(['User', 'Item', 'Rating'])
    
    n_User = len(data_sort.User.unique())
    n_Item = len(data_sort.Item.unique())
    
    return data_sort, n_User, n_Item

In [404]:
def make_R(data):
    R = data.pivot(index ='User', columns = 'Item', values = 'Rating')
    R.fillna(0, inplace = True)
    R = R.values
    return R

In [405]:
def make_Y_V_W(n_User, n_Item, stdv_Y, stdv_V, stdv_W, dimension, parameters):
    Y = stdv_Y*np.random.randn(dimension, n_User)
    V = stdv_V*np.random.randn(dimension, n_Item)
    W = stdv_W*np.random.randn(dimension, n_Item)
    parameters['Y'] = Y
    parameters['V'] = V
    parameters['W'] = W
    return parameters

In [406]:
def RMSE_calculation(original_data, predict_data):
#     I = ~np.isnan(original_data)
#     N = I.sum()
#     sqerror = abs(original_data-predict_data)**2
#     mse = sqerror[I].sum()/N
    
    xi, yi = original_data.nonzero()
    error = 0
    count = 0
    for x, y in zip(xi,yi):
        error += pow(original_data[x,y]-predict_data[x,y], 2)
        count+=1
    mse = error/count
    return np.sqrt(mse)

In [407]:
def make_sum_W(parameters, R, i, dimension):
    W = parameters['W']
    wj = R[i,:].nonzero()
#     print(W.shape)
#     print(wj[0])
    sum_W = np.zeros((10,1))
    count_W = 0
    for k in wj[0]:
#         print(k)
#         print(W[:,k].shape)
#         print(sum_W.shape)
#         print(np.array([W[:,k]]).shape)

        sum_W += np.array([W[:,k]]).T
        count_W +=1
    return sum_W, count_W, sum_W/count_W, wj

In [408]:
def gradient(parameters, i, j, R, stdv, stdv_Y, stdv_V, stdv_W,learning_rate, moment_Y, moment_V, moment_W):
    Y = parameters['Y']
    V = parameters['V']
    W = parameters['W']
    
    sum_W, count_W, cal_sum_W, wj = make_sum_W(parameters, R, i, dimension)
    
    YV_mean = np.dot(np.array([Y[:,i]])+cal_sum_W, np.array([V[:,j]]).T)
    
    difference = R[i,j] - YV_mean
    lambda_Y = (stdv/stdv_Y)**2
    lambda_V = (stdv/stdv_V)**2
    lambda_W = (stdv/stdv_W)**2
    copy_Y = Y.copy()
    copy_V = V.copy()
    
    learning_Y = (learning_rate*(difference[0,0]*V[:,j] - lambda_Y*np.abs(Y[:,i])))
    learning_V = (learning_rate*(difference[0,0]*(copy_Y[:,i] + cal_sum_W[:,0]) - lambda_V*np.abs(V[:,j])))
    
    learning_W_1 = (learning_rate*(difference[0,0]*copy_V[:,j]/count_W))
    learning_W_2 = -learning_rate*np.abs(W[:,j])
    
    learning_W = learning_W_1 + learning_W_2
    
    Y_learn = np.mean(learning_Y)
    V_learn = np.mean(learning_V)
    W_learn = np.mean(learning_W)
    
    if not (Y_learn>0):
        moment_Y[i,j]=0.0
    if not (V_learn>0):
        moment_V[i,j] = 0.0
    if not (W_learn>0):
        moment_W[i,j] = 0.0
        
    moment_Y[i,j] = momentum*moment_Y[i,j] + Y_learn
    moment_V[i,j] = momentum*moment_V[i,j] + V_learn
    moment_W[i,j] = momentum*moment_W[i,j] + np.mean(learning_W_2)
    
    Y[:,i] += moment_Y[i,j]
    V[:,j] += moment_V[i,j]
    W += np.mean(learning_W_1)
    W[:,j] += moment_W[i,j]

    parameters['Y'] = Y
    parameters['V'] = V
    parameters['W'] = W

    return parameters

In [409]:
def make_U(parameters, R, n_User, dimension):
    Y = parameters['Y']
    W = parameters['W']
    U = np.zeros(Y.shape)
    for i in range(n_User):
        sum_W, count_W, cal_W, wj = make_sum_W(parameters, R, i, dimension)
#         print(U[:,i].shape)
#         print(Y[:,i].shape)
#         print(cal_W.shape)
        U[:,i] = Y[:,i] + cal_W[:,0]
    return U
    

In [410]:
def train_and_test(parameters, epoch, n_User, n_Item, R, R_test):
    costs=[]
    test_costs = []
    for i in range(epoch):
        for j in range(n_User):
            for k in range(n_Item):
                if R[j,k]>0:
                    gradient(parameters, j, k, R, stdv, stdv_Y, stdv_V, stdv_W, learning_rate, moment_Y, moment_V, moment_W)
        Y = parameters['Y']
        V = parameters['V']
        W = parameters['W']
        
        U = make_U(parameters, R, n_User, dimension)
        
        R_predict_mean = np.dot(U.T,V)
        R_predict = np.random.normal(R_predict_mean, stdv)

        cost = RMSE_calculation(R, R_predict)
        cost_test = RMSE_calculation(R_test, R_predict)
        costs.append(cost)
        test_costs.append(cost_test)
        if (epoch%10 == 0):
            print("epoch: %d ... RMSE: %f ... test_RMSE: %f" %(i, cost, cost_test))
    return costs, test_costs, parameters

In [411]:
print("start running ------%")
data_sort, n_User, n_Item = import_data("../ml-100k/u1.base")
test_data_sort, test_n_User, test_n_Item = import_data("../ml-100k/u1.test")
R = make_R(data_sort)
R_test = make_R(test_data_sort)

moment_Y = np.zeros(R.shape)
moment_V = np.zeros(R.shape)
moment_W = np.zeros(R.shape)

make_Y_V_W(n_User, n_Item, stdv_Y, stdv_V, stdv_W, dimension, parameters)

costs, test_costs, parameters = train_and_test(parameters, epoch, n_User, n_Item, R, R_test)

plt.ylim([0.90,1.5])
plt.plot(costs, 'b', label='Train')
plt.plot(test_costs, 'r', label='Test')
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.title('PMF')
plt.minorticks_on()
plt.grid(b=True, which = 'major', color = '#666666', linestyle = '-', alpha = 0.2)
plt.legend()

plt.show()
print('----------')
print(parameters['Y'])
print('----------')
print(parameters['V'])
print('----------')
print(parameters['W'])

start running ------%
epoch: 0 ... RMSE: 1.644628 ... test_RMSE: 1.943254
epoch: 1 ... RMSE: 2.055417 ... test_RMSE: 2.378941


/home/sein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in multiply


epoch: 2 ... RMSE: nan ... test_RMSE: nan
epoch: 3 ... RMSE: nan ... test_RMSE: nan
epoch: 4 ... RMSE: nan ... test_RMSE: nan
epoch: 5 ... RMSE: nan ... test_RMSE: nan


KeyboardInterrupt: 